<a href="https://colab.research.google.com/github/iras-mpark/MLA1020/blob/main/week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
from einops import einsum

# Tensor Recap (p. 2)

### Operations on Vectors

In [4]:
x = np.array([0, 1, 10])
x

array([ 0,  1, 10])

In [5]:
# Identity: y[i] = x[i] for all i
y = einsum(x, "i -> i")
y

array([ 0,  1, 10])

In [6]:
# Sum: y += x[i] for all i
y = einsum(x, "i ->")
y

np.int64(11)

In [7]:
# Elementwise product: y[i] = x[i] * x[i] for all i
y = einsum(x, x, "i, i -> i")
y

array([  0,   1, 100])

In [8]:
# Dot product: y += x[i] * x[i] for all i
y = einsum(x, x, "i, i ->")
y

np.int64(101)

In [9]:
# Outer product: y[i][j] = x[i] * x[j] for all i, j
y = einsum(x, x, "i, j -> i j")
y

array([[  0,   0,   0],
       [  0,   1,  10],
       [  0,  10, 100]])

In [10]:
# Triple elementwise product: y[i] = x[i] * x[i] * x[i] for all i
y = einsum(x, x, x, "i, i, i -> i")
y

array([   0,    1, 1000])

In [11]:
# Triple outer product: y[i][j][k] = x[i] * x[j] * x[k] for all i, j, k
y = einsum(x, x, x, "i, j, k -> i j k")
y

array([[[   0,    0,    0],
        [   0,    0,    0],
        [   0,    0,    0]],

       [[   0,    0,    0],
        [   0,    1,   10],
        [   0,   10,  100]],

       [[   0,    0,    0],
        [   0,   10,  100],
        [   0,  100, 1000]]])

### Operations on matrices

In [12]:
x = np.array([1, 2, 3])
s = x.shape
s

(3,)

In [13]:
m = np.array([[0, 1, 2], [1, 10, 0]])
m

array([[ 0,  1,  2],
       [ 1, 10,  0]])

In [14]:
# Sum of all entries: y += m[i][j] for all i, j
y = einsum(m, "i j ->")
y

np.int64(14)

In [15]:
# Row sums: y[i] += m[i][j] for all i, j
y = einsum(m, "i j -> i")
y

array([ 3, 11])

In [16]:
# Column sums: y[j] += m[i][j] for all i, j
y = einsum(m, "i j -> j")
y

array([ 1, 11,  2])

In [17]:
# Transpose: y[j][i] = m[i][j] for all i, j
y = einsum(m, "i j -> j i")
y

array([[ 0,  1],
       [ 1, 10],
       [ 2,  0]])

In [18]:
# Matrix vector product: y[i] = m[i][j] * x[j] for all i, j
y = einsum(m, x, "i j, j -> i")
y

array([ 8, 21])

In [19]:
# Matrix-matrix product m m^T: y[i][j] = m[i][k] * m[j][k] for all i, j, k
y = einsum(m, m, "i k, j k -> i j")
y

array([[  5,  10],
       [ 10, 101]])

In [20]:
# Matrix-matrix product m^T m: y[i][j] = m[k][i] * m[k][j] for all i, j, k
y = einsum(m, m, "k i, k j -> i j")
y

array([[  1,  10,   0],
       [ 10, 101,   2],
       [  0,   2,   4]])

#Linear Regression (p. 5)

### Basic of linear regression

In [21]:
# Setup inital Data

x = np.array([[1, 2, 0], [0, -1, 1]])  # n x d 행렬  (input)
y = np.array([0, 3])  # 타깃(target)들의 n 벡터 (correct answer)
w = np.array([1, 0, 1])  # 가중치(weights)의 d 벡터 (weight)

In [22]:
# Basic Calculation

predictions = x @ w   # 곱셈 -> 예측값 n 벡터
residuals = predictions - y   # 원소별 뺄셈 -> 잔차(residual) n 벡터
losses = residuals ** 2  # 원소별 제곱
total_loss = np.sum(losses)  # 모든 원소 합

In [23]:
# Setup basic calculation to Function

def objective(w: np.ndarray) -> float:
    loss = np.sum((x @ w - y) ** 2)
    return loss

In [24]:
# Calculate loss for different Fixed Weights
loss = objective(np.array([1, 0, 1]))
print(loss)
loss = objective(np.array([1, 0, -1]))
print(loss)

# 궁극적인 목표는 objective(w)를 최소화하는 w를 찾는 것.
# 지금은: 고정된 w가 주어졌을 때, objective(w)의 값은 어떻게 될까? → 더 좋게 (작게) 만들려면 w를 어떻게 변경해야 할까?

5
17


### Example of Gradient

In [25]:
# Define 1D → 1D function
def f(x: float) -> float:
  return x ** 2

In [28]:
# -2 <= x <= 2의 범위 내에서 30개의 선형 샘플 추출 → 각 샘플에 대응되는 함수값 y 획득 가능
# []: list type
# {'x':x, 'y':y}: dictionary type
values = [{"x": x, "y": f(x)} for x in np.linspace(-2, 2, 30)]
values

[{'x': np.float64(-2.0), 'y': np.float64(4.0)},
 {'x': np.float64(-1.8620689655172413), 'y': np.float64(3.4673008323424495)},
 {'x': np.float64(-1.7241379310344827), 'y': np.float64(2.9726516052318663)},
 {'x': np.float64(-1.5862068965517242), 'y': np.float64(2.5160523186682524)},
 {'x': np.float64(-1.4482758620689655), 'y': np.float64(2.0975029726516055)},
 {'x': np.float64(-1.3103448275862069), 'y': np.float64(1.7170035671819261)},
 {'x': np.float64(-1.1724137931034484), 'y': np.float64(1.3745541022592156)},
 {'x': np.float64(-1.0344827586206897), 'y': np.float64(1.0701545778834722)},
 {'x': np.float64(-0.896551724137931), 'y': np.float64(0.8038049940546969)},
 {'x': np.float64(-0.7586206896551724), 'y': np.float64(0.5755053507728893)},
 {'x': np.float64(-0.6206896551724137), 'y': np.float64(0.38525564803804985)},
 {'x': np.float64(-0.48275862068965525), 'y': np.float64(0.23305588585017845)},
 {'x': np.float64(-0.3448275862068966), 'y': np.float64(0.11890606420927469)},
 {'x': np.flo